In [ ]:
import pandas as pd

In [ ]:
# Ratings df
ratings = pd.read_csv('goodreads_ratings.csv', index_col=[0])

display(ratings.head())
ratings.shape

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


(5976479, 3)

In [ ]:
# Books df
books = pd.read_csv('goodreads_books.csv')
books = books[["book_id", "authors", "title"]]
print("shape: ", books.shape)
books.head()


shape:  (10000, 3)


,book_id,authors,title
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)"
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...
2,3,Stephenie Meyer,"Twilight (Twilight, #1)"
3,4,Harper Lee,To Kill a Mockingbird
4,5,F. Scott Fitzgerald,The Great Gatsby


In [ ]:
# Ratings of uusers with at least 120 ratings
counts1 = ratings['user_id'].value_counts()
ratings = ratings[ratings['user_id'].isin(counts1[counts1 >= 120].index)]

# Ratings of Books with at least 80 ratings
counts = ratings['rating'].value_counts()
ratings = ratings[ratings['rating'].isin(counts[counts >= 80].index)]

In [5]:
ratings.head()
# ratings.shape

,user_id,book_id,rating
12,4,70,4
13,4,264,3
14,4,388,4
15,4,18,5
16,4,27,5


In [ ]:
# Merge the two dataframes
combine_book_rating = pd.merge(ratings, books, on='book_id')
columns = ['authors']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,user_id,book_id,rating,title
0,4,70,4,"Ender's Game (Ender's Saga, #1)"
1,4,264,3,The Sun Also Rises
2,4,388,4,"Hatchet (Brian's Saga, #1)"
3,4,18,5,Harry Potter and the Prisoner of Azkaban (Harr...
4,4,27,5,Harry Potter and the Half-Blood Prince (Harry ...


In [7]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])

book_ratingCount = (combine_book_rating.
     groupby(['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

book_ratingCount.head()

,title,totalRatingCount
0,"Angels (Walsh Family, #3)",114
1,#GIRLBOSS,61
2,'Salem's Lot,2452
3,"'Tis (Frank McCourt, #2)",383
4,"1,000 Places to See Before You Die",199


In [8]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,user_id,book_id,rating,title,totalRatingCount
0,4,70,4,"Ender's Game (Ender's Saga, #1)",4281
1,4,264,3,The Sun Also Rises,2286
2,4,388,4,"Hatchet (Brian's Saga, #1)",1199
3,4,18,5,Harry Potter and the Prisoner of Azkaban (Harr...,6817
4,4,27,5,Harry Potter and the Half-Blood Prince (Harry ...,6536


In [9]:
popularity_threshold = 2500
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,user_id,book_id,rating,title,totalRatingCount
0,4,70,4,"Ender's Game (Ender's Saga, #1)",4281
3,4,18,5,Harry Potter and the Prisoner of Azkaban (Harr...,6817
4,4,27,5,Harry Potter and the Half-Blood Prince (Harry ...,6536
5,4,21,5,Harry Potter and the Order of the Phoenix (Har...,6596
6,4,2,5,Harry Potter and the Sorcerer's Stone (Harry P...,8841


In [11]:
from scipy.sparse import csr_matrix
rating_popular_book = rating_popular_book.drop_duplicates(['user_id', 'title'])
rating_popular_book.shape


(623719, 5)

In [13]:
# Get the pivot matrix and store it in a csr matrix

rating_popular_book_pivot = rating_popular_book.pivot(index = 'title', columns = 'user_id', values = 'rating').fillna(0)
rating_popular_book_matrix = csr_matrix(rating_popular_book_pivot.values)

rating_popular_book_pivot.head()



user_id,4,7,9,10,11,22,23,24,27,28,...,53390,53400,53401,53403,53409,53411,53413,53419,53422,53424
title,,,,,,,,,,,,,,,,,,,,,
1984,4.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,3.0,5.0,3.0,0.0,0.0,2.0,0.0,0.0
A Christmas Carol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"A Clash of Kings (A Song of Ice and Fire, #2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Clockwork Orange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"A Feast for Crows (A Song of Ice and Fire, #4)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


In [14]:
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto')
model_knn.fit(rating_popular_book_matrix)

NearestNeighbors(metric='cosine')

##Testing

In [15]:
import numpy as np

In [27]:
query_index = np.random.choice(rating_popular_book_pivot.shape[0])
print(query_index)
print(rating_popular_book_pivot.index[query_index])


109
The Handmaid's Tale


In [45]:
book = input("Enter the exact name of the book you want to get recommendations for: ")
rating_popular_book_pivot.loc[book].values.reshape(1, -1)

# distances, indices = model_knn.kneighbors(rating_popular_book_pivot.loc[book].values.reshape(1, -1), n_neighbors = 6)

# print(indices)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [28]:
distances, indices = model_knn.kneighbors(rating_popular_book_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [29]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_popular_book_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_book_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Handmaid's Tale:

1: 1984, with distance of 0.6362589552000611:
2: Memoirs of a Geisha, with distance of 0.6452778187981918:
3: To Kill a Mockingbird, with distance of 0.6482055113412749:
4: Brave New World, with distance of 0.6536931421330112:
5: The Poisonwood Bible, with distance of 0.6585409808008259:


In [30]:
# save rating_popular_book_pivot and knn model
rating_popular_book.to_csv('output/rating_popular_book.csv')

import pickle
with open('output/model_knn.pkl', 'wb') as file:
    pickle.dump(model_knn, file)